In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Fibromyalgia/GSE67311'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Peripheral Blood Gene Expression in Fibromyalgia Patients Reveals  Potential Biological Markers and Physiological Pathways"
!Series_summary	"Fibromyalgia (FM) is a common pain disorder characterized by dysregulation in the processing of pain. Although FM has similarities with other rheumatologic pain disorders, the search for objective markers has not been successful. In the current study we analyzed gene expression in the whole blood of 70 fibromyalgia patients and 70 healthy matched controls. Global molecular profiling revealed an upregulation of several inflammatory molecules in FM patients and downregulation of specific pathways related to hypersensitivity and allergy. There was a differential expression of genes in known pathways for pain processing, such as glutamine/glutamate signaling and axonal development. We also identified a panel of candidate gene expression-based classifiers that could establish an objective blood-based molecular dia

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

import numpy as np

# 1. Gene Expression Data Availability
is_gene_available = True  # Based on the provided information, this dataset likely contains gene expression data.

# 2. Variable Availability and Data Type Conversion
for key, values in {
    0: ['diagnosis: fibromyalgia', 'diagnosis: healthy control'], 
    2: ['fiqr score: 2.2', 'fiqr score: 22.8', ...],  # Sample example
    4: ['migraine: Yes', 'migraine: No', np.nan]
}.items():
    if key == 0:
        trait_row = key
    if key == 2:
        age_row = key
    if key == 4:
        gender_row = key

# Data type conversion functions
def convert_trait(value):
    try:
        value = value.split(': ')[1].lower()
    except IndexError:
        return None
    if value == 'fibromyalgia':
        return 1
    elif value == 'healthy control':
        return 0
    else:
        return None

def convert_age(value):
    try:
        value = value.split(': ')[1]
        return float(value)
    except (IndexError, ValueError):
        return None

def convert_gender(value):
    try:
        value = value.split(': ')[1].lower()
    except IndexError:
        return None
    if value == 'male':
        return 1
    elif value == 'female':
        return 0
    else:
        return None

# 3. Save Metadata
save_cohort_info('GSE67311', './preprocessed/Fibromyalgia/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Fibromyalgia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Fibromyalgia/trait_data/GSE67311.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1644447': [0.0, 8.5, nan], 'GSM1644448': [0.0, -2.0, nan], 'GSM1644449': [0.0, 9.8, nan], 'GSM1644450': [0.0, 0.5, nan], 'GSM1644451': [0.0, -1.0, nan], 'GSM1644452': [0.0, -0.5, nan], 'GSM1644453': [0.0, 2.2, nan], 'GSM1644454': [0.0, -2.0, nan], 'GSM1644455': [0.0, -2.0, nan], 'GSM1644456': [0.0, 15.3, nan], 'GSM1644457': [0.0, 4.0, nan], 'GSM1644458': [0.0, 0.5, nan], 'GSM1644459': [0.0, 29.3, nan], 'GSM1644460': [0.0, 27.2, nan], 'GSM1644461': [0.0, 27.2, nan], 'GSM1644462': [0.0, -1.0, nan], 'GSM1644463': [0.0, 5.0, nan], 'GSM1644464': [0.0, 1.0, nan], 'GSM1644465': [0.0, -1.0, nan], 'GSM1644466': [0.0, -0.5, nan], 'GSM1644467': [0.0, 4.0, nan], 'GSM1644468': [0.0, 2.5, nan], 'GSM1644469': [0.0, 8.5, nan], 'GSM1644470': [0.0, -0.5, nan], 'GSM1644471': [0.0, 5.0, nan], 'GSM1644472': [0.0, 1.0, nan], 'GSM1644473': [0.0, 3.0, nan], 'GSM1644474': [0.0, 4.0, nan], 'GSM1644475': [0.0, -0.5, nan], 'GSM1644476': [0.0, -1.5, nan], 'GSM1644477': [0.0, 1.3, nan], 'GSM1644478': [0.0, 2.5

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['7892501', '7892502', '7892503', '7892504', '7892505', '7892506',
       '7892507', '7892508', '7892509', '7892510', '7892511', '7892512',
       '7892513', '7892514', '7892515', '7892516', '7892517', '7892518',
       '7892519', '7892520'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001005240,NM_001004195,NM_001005484,BC136848,BC136907', 'BC118988,AL137655', 'NM_001005277,NM_001005221,NM_001005224,NM_001005504,BC137547'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [53049.0, 63015.0, 69091.0, 334129.0, 367659.0], 'RANGE_STOP': [54936.0, 63887.0, 70008.0, 334296.0, 368597.0], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', '---', 'NM_001005240 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// NM_001004195 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 /// NM_001005484 // OR4F5 //

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the key names storing relevant identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'gene_assignment'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to obtain gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
